In [22]:
import tensorflow as tf
import tensorflow_hub as hub
#library used to perform transfer learning
import os
from typing import Tuple

In [23]:
"""Creates the data generators to be used in training the network
Args
    dataset: location of the dataset
    image_dim: scale to this image dimension
    batch_size: batch size
Returns
    train_generator: ImageDataGenerator
tf.keras.preprocessing : slow in training
"""
def load_generators(dataset_dir: str, image_dim: Tuple[int, int], batch_size: int):
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        dataset_dir,
        shuffle=True,
        color_mode="rgb",
        class_mode="categorical",
        target_size=image_dim,
        batch_size=batch_size)
    # it takes input images and outputs augmented images

    print('created dataset for {}'.format(dataset_dir))

    return train_generator

#This function performs the data augmentation

In [24]:
BATCH_SIZE = 24
IMG_HEIGHT = 299
IMG_WIDTH = 299
#This is the default input size for Inceptionv3 architecture

# set the train and validation datasets
train_dir = '/kaggle/input/coffee-dataset/coffee-datasets/symptom/train'
validation_dir = '/kaggle/input/coffee-dataset/coffee-datasets/symptom/val'

# load the data generators
train_datagen = load_generators(train_dir, (IMG_HEIGHT, IMG_WIDTH), BATCH_SIZE)

validation_datagen = load_generators(validation_dir, (IMG_HEIGHT, IMG_WIDTH), BATCH_SIZE)
#These new datasets are new transformed images produced using ImageDataGenerator class required for transfer learning.

Found 1539 images belonging to 5 classes.
created dataset for /kaggle/input/coffee-dataset/coffee-datasets/symptom/train
Found 335 images belonging to 5 classes.
created dataset for /kaggle/input/coffee-dataset/coffee-datasets/symptom/val


In [25]:
INCEPTIONV3_TFHUB = 'https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4'
#Getting the InceptionV3 architecture model that is 47 layer deep

In [26]:
input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)
num_classes = train_datagen.num_classes
#declaring the classes and input shape

# fetch the feature extractor from the tf_hub
feature_extractor = hub.KerasLayer(INCEPTIONV3_TFHUB, input_shape=input_shape)
#used to gather the transfer learning model

# make the feature extractor trainable
feature_extractor.trainable = True
# create the sequential model
model = tf.keras.Sequential([
    feature_extractor, 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
])
#creating a sequential 
#contains the inceptionv3 model along with a flatten and an Dense layer added to model.

In [27]:
# print the summary of the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 2048)              21802784  
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 10245     
Total params: 21,813,029
Trainable params: 21,778,597
Non-trainable params: 34,432
_________________________________________________________________


In [28]:
# compiling the model
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9),
    #stochastic gradient descent optimizer
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [29]:
# training the model
model.fit(
    train_datagen,
    epochs=20,
    steps_per_epoch=train_datagen.samples//train_datagen.batch_size,
    validation_data=validation_datagen,)

Train for 64 steps, validate for 14 steps
Epoch 1/20
64/64 [==============================] - 26s 400ms/step - loss: 0.7483 - accuracy: 0.8205 - val_loss: 1.8821 - val_accuracy: 0.5493
Epoch 2/20
64/64 [==============================] - 15s 229ms/step - loss: 0.4409 - accuracy: 0.9360 - val_loss: 1.8344 - val_accuracy: 0.6567
Epoch 3/20
64/64 [==============================] - 15s 227ms/step - loss: 0.3142 - accuracy: 0.9782 - val_loss: 0.4931 - val_accuracy: 0.9194
Epoch 4/20
64/64 [==============================] - 15s 228ms/step - loss: 0.2792 - accuracy: 0.9901 - val_loss: 0.4848 - val_accuracy: 0.9284
Epoch 5/20
64/64 [==============================] - 15s 227ms/step - loss: 0.2562 - accuracy: 0.9947 - val_loss: 0.3788 - val_accuracy: 0.9672
Epoch 6/20
64/64 [==============================] - 15s 229ms/step - loss: 0.2391 - accuracy: 0.9987 - val_loss: 0.3507 - val_accuracy: 0.9701
Epoch 7/20
64/64 [==============================] - 14s 226ms/step - loss: 0.2359 - accuracy: 1.0000

In [30]:
# evaluate the model
loss, accuracy = model.evaluate(validation_datagen)
# train accuracy
train_loss, train_accuracy = model.evaluate(train_datagen)

65/65 [==============================] - 6s 87ms/step - loss: 0.2339 - accuracy: 1.0000


In [31]:
"Trained Model for {} epochs, train accuracy: {:5.2f}%, test accuracy: {:5.2f}%".format(2, 100*train_accuracy, 100*accuracy)

'Trained Model for 2 epochs, train accuracy: 100.00%, test accuracy: 97.61%'